In [31]:
import numpy as np
import os

In [32]:
REG_FILE = {'00000' : '0000000000000000000000000000000000000000000000000000000000111111',
            '00001' : '0000000000000000000000000000000000000000000000000000000000111011',
            '00010' : '0000000000000000000000000000000000000000000000000000000000100111',
            '00011' : '0000000000000000000000000000000000000000000000000000000000111111',
            '00100' : '0000000000000000000000000000000000000000000000000000000000110101',
            '00101' : '0000000000000000000000000000000000000000000000000000000000110110',
            '00110' : '0000000000000000000000000000000000000000000000000000000000010111',
            '00111' : '0000000000000000000000000000000000000000000000000000000001011011',
            '01000' : '0000000000000000000000000000000000000000000000000000000000110111',
            '01001' : '0000000000000000000000000000000000000000000000000000000000111111',
            '01010' : '0000000000000000000000000000000000000000000000000000000000110001',
            '01011' : '0000000000000000000000000000000000000000000000000000000000000111',
            '01100' : '0000000000000000000000000000000000000000000000000000000011110111',
            '01101' : '0000000000000000000000000000000000000000000000000000000000110011',
            '01110' : '0000000000000000000000000000000000000000000000000000000000111101',
            '01111' : '0000000000000000000000000000000000000000000000000000000000011111',
            '10000' : '0000000000000000000000000000000000000000000000000000000000111110',
            '10001' : '0000000000000000000000000000000000000000000000000000000000011110',
            '10010' : '0000000000000000000000000000000000000000000000000000000000111001',
            '10011' : '0000000000000000000000000000000000000000000000000000000001011111',
            '10100' : '0000000000000000000000000000000000000000000000000000000000110111',
            '10101' : '0000000000000000000000000000000000000000000000000000000000110101',
            '10110' : '0000000000000000000000000000000000000000000000000000000000110000',
            '10111' : '0000000000000000000000000000000000000000000000000000000000100111',
            '11000' : '0000000000000000000000000000000000000000000000000000000000110101',
            '11001' : '0000000000000000000000000000000000000000000000000000000001011011',
            '11010' : '0000000000000000000000000000000000000000000000000000000000110101',
            '11011' : '0000000000000000000000000000000000000000000000000000000000110111',
            '11100' : '0000000000000000000000000000000000000000000000000000000000111111',
            '11101' : '0000000000000000000000000000000000000000000000000000000000110111',
            '11110' : '0000000000000000000000000000000000000000000000000000000000110101',
            '11111' : '0000000000000000000000000000000000000000000000000000000000110000',
            }

MEM_VALUE={
            '1' : '0000000000000000000000000000000000000000000000000000000000000000',
            '2' : '0000000000000000000000000000000000000000000000000000000000000011',
            '3' : '0000000000000000000000000000000000000000000000000000000000000010',
            '4' : '0000000000000000000000000000000000000000000000000000000000000011',
            '5' : '0000000000000000000000000000000000000000000000000000000000000100',
            '6' : '0000000000000000000000000000000000000000000000000000000000000101',
            '7' : '0000000000000000000000000000000000000000000000000000000000000110',
            '8' : '0000000000000000000000000000000000000000000000000000000000000111',
            '9' : '0000000000000000000000000000000000000000000000000000000000001000',
            '9' : '0000000000000000000000000000000000000000000000000000000000001001',
            '10': '0000000000000000000000000000000000000000000000000000000000111111',
}

In [33]:
# Get opcodes
opCodes = {}
def getOPCode():
    with open('inst_format.txt',"r") as op:
        data = op.readlines()
    for s in data:
        if s is None:
            continue
        t = s.split()
        if t[0] == "//":
            continue
        opCodes[t[1]] = t[0]
    return opCodes

In [34]:
# Convert instructions to binary 
def ins_Binary(testcase):
    with open(testcase,'r') as tc:
        opCodes=getOPCode()
        ins=tc.readlines()
        insbinarylist=[]
        for i in ins:
            i.strip()
            ins_split=i.split()
            for i in range(len(ins_split)):
                if i==0:
                    y=opCodes[ins_split[0]]+" "
                else:
                    if(ins_split[i][0]=='#'):
                        z=ins_split[i].replace('#',"")
                        y=y+z+' '
                    else:
                        z=ins_split[i].replace('R',"")
                        z=str(bin(int(z))[2:])
                        y=y+z.zfill(5)+' '
            insbinarylist.append(y)
    return insbinarylist

In [35]:
# Make array of instructions in corresponding FU
# Resolve structural hazards
def makeList(inst_FU):
    print(inst_FU)
    insPacket=np.zeros((30,6),dtype=int)
    add=0
    mul=0
    fadd=0
    fmul=0
    logic=0
    memory=0
    for i in inst_FU:
        ins=i.strip().split(" ")
        if(ins[0] in ['000000','000001']):
            insPacket[add][0]=inst_FU.index(i)+1
            add=add+1
        elif(ins[0]=='000010'):
            insPacket[mul][1]=inst_FU.index(i)+1
            mul=mul+1
        elif(ins[0]=='000011'):
            insPacket[fadd][2]=inst_FU.index(i)+1
            fadd=fadd+1
        elif(ins[0]=='000101'):
            insPacket[fmul][3]=inst_FU.index(i)+1
            fmul=fmul+1
        elif(ins[0] in ['100000','100001','100110','100010','100011','100111','100100','100101']):
            insPacket[logic][4]=inst_FU.index(i)+1
            logic=logic+1
        elif(ins[0] in ['000110','000111']):
            insPacket[memory][5]=inst_FU.index(i)+1
            memory=memory+1   
    return insPacket

In [36]:
# Check data hazards
def checkDependencies(insPacketlist):
    dependencieslist=[]
    for i in range(len(insPacketlist)):
        ins=insPacketlist[i].strip().split(" ")
        if(ins[0] == '000110'):
            w1=ins[1]
            r11=None
            r12=None
        elif(ins[0] == '000111'):
            w1=None
            r11=ins[1]
            r12=None  
        else:
            w1,r11,r12=ins[1],ins[2],ins[3]
        for j in range(i+1,len(insPacketlist)):
            ins=insPacketlist[j].strip().split(" ")
            if(ins[0] == '000110'):
                w2=ins[1]
                r21=None
                r22=None
            elif(ins[0] == '000111'):
                w2=None
                r21=ins[1]
                r22=None
            else:
                w2,r21,r22=ins[1],ins[2],ins[3]  
            if(w1==r21 or w1==r22 or r11==w2 or r12==w2 or w1==w2):
                    dependencieslist.append([i+1,j+1])
    return dependencieslist

In [37]:
# Update the array to resolve hazards
def changeList(insPacketlist,dependencieslist):
    for i in dependencieslist:
        f1=i[0]
        f2=i[1]
        index1=np.where(insPacketlist==f1)
        index2=np.where(insPacketlist==f2)
        list=[[0],[1],[2],[3],[4],[5]]
        #Add
        if(index1[1]==list[0]):
            insPacketlist[(index2)]=0
            i0=index1[0]+5
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
        #Mul
        elif(index1[1]==list[1]):
            insPacketlist[(index2)]=0
            i0=index1[0]+8
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
        #FAdd
        elif(index1[1]==list[2]):
            insPacketlist[(index2)]=0
            i0=index1[0]+12
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
        #FMul
        elif(index1[1]==list[3]):
            insPacketlist[(index2)]=0
            i0=index1[0]+14
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
        #LU
        elif(index1[1]==list[4]):
            insPacketlist[(index2)]=0
            i0=index1[0]+1
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
        #Memory
        elif(index1[1]==list[5]):
            insPacketlist[(index2)]=0
            i0=index1[0]+1
            i1=index2[1]
            newindex=(i0,i1)
            while True:
                if(insPacketlist[(newindex)]==0):
                    insPacketlist[(newindex)]=f2
                    break
                else:
                    i0=i0+1
                    insPacketlist[(i0,i1)]=f2
                    break
    return insPacketlist

In [38]:
# Make sets of instructions (parallel issue)
def makepackets(insPacketList):
    newlist=insPacketList.take(list(range(0,20)),axis=0)
    packetSet=[]
    for ele in newlist:
        for i in ele:
            if(i==0):
                packetSet.append('NOP')
            else:
                packetSet.append(binarylist[i-1])
    return packetSet

In [39]:
binarylist=ins_Binary('programs/p1.txt')
insPacketlist=makeList(binarylist)
dependencieslist=checkDependencies(binarylist)
insPacketlist=changeList(insPacketlist,dependencieslist)
packetSet=makepackets(insPacketlist)

['000101 11001 10101 11000 ', '000000 00010 00001 00011 ', '000001 00100 00010 00001 ', '000111 00111 7 ', '000010 01000 10011 01111 ', '100111 01100 10011 00001 ', '000011 10111 00010 11000 ', '100000 01000 00010 10101 ', '100001 00001 01100 10110 ', '000110 00100 2 ']


In [ ]:
def endtb(tb_str):
        tb_str = tb_str + "end\n"
        tb_str = tb_str + "initial begin\n"
        tb_str = tb_str + "$monitor(\"%"+"b\",out);\n"
        tb_str = tb_str + "end\n"
        tb_str = tb_str + "endmodule\n"
        file1 = open("test.v","w")
        file1.write(tb_str)
        file1.close()
        os.system("iverilog test.v")
        os.system("vvp a.out > output.txt")
        file2 = open("output.txt","r")
        result = file2.readlines()
        finaloutfile=open("finalout.txt","a+")
        finaloutfile.write(str(result)+'\n')
        file2.close()
        REG_FILE[rdes] = result[0][:-1]
        print('Result:',result[0][:-1])
sets=[]
sets = [packetSet[6*i:(6*i)+6] for i in range((len(packetSet) + 6 - 1) // 6 )]
for s in sets:
    for ins in s:
        ins=ins.split(' ')
        if(ins[0]=='NOP'):
            opcode=ins[0]
            rdes=  -1
            rsrc1= -1
            rsrc2= -1
        elif(ins[0] in ['000111','000110']):
            opcode=ins[0]
            rdes=ins[1]
            mem=ins[2]	
        else:
            opcode=ins[0]
            rdes=  ins[1]
            rsrc1= ins[2]
            rsrc2= ins[3]
        #Add
        if(opcode=='000000'):
            print('Addition ')
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"ADD.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nreg cin;\nwire [63:0] out;\nwire cout;\n"
            tb_str = tb_str + "adder k(n1, n2, cin, out, cout);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+";\n cin = 1'b0;\n"
            endtb(tb_str)
        #Subtract
        elif(opcode=='000001'):
            print('Subtraction')
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"ADD.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nreg cin;\nwire [63:0] out;\nwire cout;\n"
            tb_str = tb_str + "SUB k(n1, n2, cin, out, cout);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+";\n cin = 1'b1;\n"
            endtb(tb_str)
        #Mul
        elif(opcode=='000010'):
            print('Multiplication')
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"MUL.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nreg cin;\nwire [63:0] out;\n"
            tb_str = tb_str + "Multiplier k(n1, n2, out);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+";\n"
            endtb(tb_str)
        #FAdd
        elif(opcode=='000011'):
            print('Floating Point Additon')
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"FADD.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nwire [63:0] out;\n"
            tb_str = tb_str + "FLA k(n1, n2, out);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+";\n"
            endtb(tb_str)
        #FMul
        elif(opcode=='000101'):
            print('Floating Point Multiplication')
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"FMUL.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nwire [63:0] out;\n"
            tb_str = tb_str + "FLM k(n1, n2, out);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+";\n"
            endtb(tb_str)
        #LU
        elif (opcode in ['100000','100001','100110','100010','100011','100111','100100','100101']):
            logic=[k for k, v in opCodes.items() if v == opcode]
            print('Logic',logic)
            print('A :  ',REG_FILE[rsrc1])
            print('B :  ',REG_FILE[rsrc2])
            tb_str = "`include \"LU.v\"\n"
            tb_str = tb_str + "module test;\n"
            tb_str = tb_str + "reg [63:0] n1;\nreg [63:0] n2;\nreg [5:0] opcode;\nwire [63:0] out;\n"
            tb_str = tb_str + "logicUnit k(n1, n2, opcode ,out);\n"
            tb_str = tb_str + "initial begin\n"
            tb_str = tb_str + "#0 n1 = 64'b"+REG_FILE[rsrc1]+"; n2 = 64'b"+REG_FILE[rsrc2]+"; opcode=6'b"+opcode+";\n"
            endtb(tb_str)
        #Load
        elif(opcode == '000110'):
            print('Load')          
            print('Destination : ',REG_FILE[rdes])
            print('Memory : ',MEM_VALUE[mem])
            REG_FILE[rdes]=MEM_VALUE[mem]
            print('After Loading')
            print('Destination : ',REG_FILE[rdes])
        #Store
        elif(opcode == '000111'):
            print('Store')
            print('Destination : ',REG_FILE[rdes])
            print('Memory :',MEM_VALUE[mem])
            MEM_VALUE[mem]=REG_FILE[rdes]
            print('After Storing')
            print('Memory :',MEM_VALUE[mem])
        elif (opcode=='NOP'):
            pass